In [1]:
# Magics



In [2]:
# Imports

In [3]:
import argparse
import logging
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam
from pyro.util import ignore_jit_warnings

In [4]:
# Settings

In [5]:

emission_matrix_path = Path("/nic/emission_matrix.npy")
dataset_path = Path("/nic/sequences.npy")


In [6]:
# Code

In [7]:
# Dataset loading
def load_sequences(dataset_path):
    sequences = {"train": {}}
    
#     sequence_list = [torch.Tensor(x) for x in np.load(str(dataset_path))]
    
    sequences_tensor = torch.Tensor(np.load(str(dataset_path))).to(torch.int)
    
    
    sequences["train"]["sequence_lengths"] = torch.Tensor(np.array([len(sequence) for sequence in sequences_tensor])).to(torch.int)
    sequences["train"]["sequences"] = sequences_tensor
    return sequences

In [8]:
def emission(x):
    return None

In [9]:
def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        
    data_dim = 7
    
#     print(sequences)
#     print(lengths)
#     print(num_sequences)
#     print(max_length)


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
                                  .to_event(1))
        
    with pyro.plate("sequences", num_sequences, batch_size, dim=-2) as batch:
        lengths = lengths[batch]
        
        x = (6* torch.ones(batch_size)).to(torch.int)
        y = (6 * torch.ones(batch_size)).to(torch.int)
        
        for t in pyro.markov(range(max_length if args.jit else lengths.max())):
            with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
#                 print(f"prob_x: {x} {x} {probs_x[x]}")
                x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
                                infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x} {x} {probs_x[x].shape}")
                
                # Note that since each tone depends on all tones at a previous time step
                # the tones at different time steps now need to live in separate plates.
                with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
                    p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
                    y = pyro.sample("y_{}".format(t),
                                    dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
                                    obs=sequences[batch, t],
                                   )

In [22]:
def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True, prediction=False):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        
    data_dim = 7
    
#     print(sequences)
#     print(lengths)
#     print(num_sequences)
#     print(max_length)


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
                                  .to_event(1))

    tones_plate = pyro.plate("tones", data_dim, dim=-1)


    for i in pyro.plate("sequences", len(sequences), batch_size):
        length = lengths[i]
        sequence = sequences[i, :length]
        if prediction:
            print(f"{i} {length} {sequence}")
        x = 6
        for t in pyro.markov(range(length)):
            # On the next line, we'll overwrite the value of x with an updated
            # value. If we wanted to record all x values, we could instead
            # write x[t] = pyro.sample(...x[t-1]...).
            x = pyro.sample("x_{}_{}".format(i, t), dist.Categorical(probs_x[x]),
                            infer={"enumerate": "parallel"})
            with tones_plate:
                p_y_t = emission_matrix[x]

                pyro.sample("y_{}_{}".format(i, t), 
                            dist.Categorical(p_y_t),
                            obs=sequence[t])
        
#         for t in pyro.markov(range(max_length if args.jit else lengths.max())):
#             with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
# #                 print(f"prob_x: {x}")
#                 x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
#                                 infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x.shape}")
                
#                 # Note that since each tone depends on all tones at a previous time step
#                 # the tones at different time steps now need to live in separate plates.
#                 with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
#                     p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
#                     y = pyro.sample("y_{}".format(t),
#                                     dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
#                                     obs=sequences[batch, t],
#                                    )

In [26]:
def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True, prediction=False):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        
    data_dim = 7
    
#     print(sequences)
#     print(lengths)
#     print(num_sequences)
#     print(max_length)


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
                                  .to_event(1))

    tones_plate = pyro.plate("tones", data_dim, dim=-1)


    for i in pyro.plate("sequences", len(sequences)):
        length = lengths[i]
        sequence = sequences[i, :length]
        if prediction:
            print(f"{i} {length} {sequence}")
        x = 6
        for t in pyro.markov(range(length)):
            # On the next line, we'll overwrite the value of x with an updated
            # value. If we wanted to record all x values, we could instead
            # write x[t] = pyro.sample(...x[t-1]...).
            x = pyro.sample("x_{}_{}".format(i, t), dist.Categorical(probs_x[x]),
                            infer={"enumerate": "parallel"})
            with tones_plate:
                p_y_t = emission_matrix[x]

                pyro.sample("y_{}_{}".format(i, t), 
                            dist.Categorical(p_y_t),
                            obs=sequence[t])
        
#         for t in pyro.markov(range(max_length if args.jit else lengths.max())):
#             with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
# #                 print(f"prob_x: {x}")
#                 x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
#                                 infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x.shape}")
                
#                 # Note that since each tone depends on all tones at a previous time step
#                 # the tones at different time steps now need to live in separate plates.
#                 with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
#                     p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
#                     y = pyro.sample("y_{}".format(t),
#                                     dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
#                                     obs=sequences[batch, t],
#                                    )

In [27]:
# Utility function to print latent sites' quantile information.
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

In [28]:
def main(args):
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')

    logging.info('Loading data')
    data = load_sequences(args.dataset_path)

    logging.info('-' * 40)
    model = model_0
    logging.info('Training {} on {} sequences'.format(
        model.__name__, len(data['train']['sequences'])))
    
    sequences = data['train']['sequences']
#     print(sequences)
    lengths = data['train']['sequence_lengths']
#     print(lengths)
    
    emission_matrix_np = np.load(args.emission_matrix_path)
    emission_matrix = torch.Tensor(emission_matrix_np)
    # find all the notes that are present at least once in the training set
#     present_notes = ((sequences == 1).sum(0).sum(0) > 0)
    # remove notes that are never played (we remove 37/88 notes)
#     sequences = sequences[..., present_notes]

    if args.truncate:
        lengths = lengths.clamp(max=args.truncate)
        sequences = sequences[:, :args.truncate]
    num_observations = float(lengths.sum())
    pyro.set_rng_seed(args.seed)
    pyro.clear_param_store()

    # We'll train using MAP Baum-Welch, i.e. MAP estimation while marginalizing
    # out the hidden state x. This is accomplished via an automatic guide that
    # learns point estimates of all of our conditional probability tables,
    # named probs_*.
    guide = AutoDelta(poutine.block(model, expose_fn=lambda msg: msg["name"].startswith("probs_")))

    # To help debug our tensor shapes, let's print the shape of each site's
    # distribution, value, and log_prob tensor. Note this information is
    # automatically printed on most errors inside SVI.
#     if args.print_shapes:
    first_available_dim = -2 if model is model_0 else -3
    guide_trace = poutine.trace(guide).get_trace(
        sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
    model_trace = poutine.trace(
        poutine.replay(poutine.enum(model, first_available_dim), guide_trace)).get_trace(
        sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
    logging.info(model_trace.format_shapes())
    print(model_trace.format_shapes())

    # Enumeration requires a TraceEnum elbo and declaring the max_plate_nesting.
    # All of our models have two plates: "data" and "tones".
    optim = Adam({'lr': args.learning_rate})
    if args.tmc:
        if args.jit:
            raise NotImplementedError("jit support not yet added for TraceTMC_ELBO")
        elbo = TraceTMC_ELBO(max_plate_nesting=1 if model is model_0 else 2)
        tmc_model = poutine.infer_config(
            model,
            lambda msg: {"num_samples": args.tmc_num_samples, "expand": False} if msg["infer"].get("enumerate", None) == "parallel" else {})  # noqa: E501
        svi = SVI(tmc_model, guide, optim, elbo)
    else:
        Elbo = JitTraceEnum_ELBO if args.jit else TraceEnum_ELBO
        elbo = Elbo(max_plate_nesting= 2,
                    strict_enumeration_warning=True,
                    jit_options={"time_compilation": args.time_compilation})
        svi = SVI(model, guide, optim, elbo)

    # We'll train on small minibatches.
    logging.info('Step\tLoss')
    for step in range(args.num_steps):
        print(f"\tTraining step: {step}")
        loss = svi.step(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
        logging.info('{: >5d}\t{}'.format(step, loss / num_observations))
        print('\t{: >5d}\t{}'.format(step, loss / num_observations))
        
    return svi, model, guide, sequences, lengths, emission_matrix

    from pyro.infer import Predictive


    num_samples = 1000
    predictive = Predictive(model, guide=guide, num_samples=num_samples)
    svi_samples = {k: v.detach().cpu().numpy()
               for k, v in predictive(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size, prediction=True).items()
               if k != "obs"}
    
    print(svi_samples)
    
    for site, values in summary(svi_samples).items():
        print("Site: {}".format(site))
        print(values, "\n")
        

        
    print(pyro.param("probs_x").item())

    if args.jit and args.time_compilation:
        logging.debug('time to compile: {} s.'.format(elbo._differentiable_loss.compile_time))

    # We evaluate on the entire training dataset,
    # excluding the prior term so our results are comparable across models.
    train_loss = elbo.loss(model, guide, sequences, lengths, emission_matrix, args, include_prior=False)
    logging.info('training loss = {}'.format(train_loss / num_observations))
    print('training loss = {}'.format(train_loss / num_observations))

    # We expect models with higher capacity to perform better,
    # but eventually overfit to the training set.
    capacity = sum(value.reshape(-1).size(0)
                   for value in pyro.get_param_store().values())
    logging.info('{} capacity = {} parameters'.format(model.__name__, capacity))
    print('{} capacity = {} parameters'.format(model.__name__, capacity))


In [29]:
# Content

In [34]:

assert pyro.__version__.startswith('1.6.0')
parser = argparse.ArgumentParser(description="MAP Baum-Welch learning Bach Chorales")

parser.add_argument("-n", "--num-steps", default=100, type=int)
parser.add_argument("-b", "--batch-size", default=1, type=int)
parser.add_argument("-d", "--hidden-dim", default=16, type=int)
parser.add_argument("-nn", "--nn-dim", default=48, type=int)
parser.add_argument("-nc", "--nn-channels", default=2, type=int)
parser.add_argument("-lr", "--learning-rate", default=0.05, type=float)
parser.add_argument("-t", "--truncate", type=int)
parser.add_argument("-p", "--print-shapes", action="store_true")
parser.add_argument("--seed", default=0, type=int)
parser.add_argument('--cuda', action='store_true')
parser.add_argument('--jit', action='store_true')
parser.add_argument('--time-compilation', action='store_true')
parser.add_argument('-rp', '--raftery-parameterization', action='store_true')
parser.add_argument('--tmc', action='store_true',
                    help="Use Tensor Monte Carlo instead of exact enumeration "
                         "to estimate the marginal likelihood. You probably don't want to do this, "
                         "except to see that TMC makes Monte Carlo gradient estimation feasible "
                         "even with very large numbers of non-reparametrized variables.")
parser.add_argument('--tmc-num-samples', default=10, type=int)

parser.add_argument("--dataset_path")
parser.add_argument("--emission_matrix_path")



_StoreAction(option_strings=['--emission_matrix_path'], dest='emission_matrix_path', nargs=None, const=None, default=None, type=None, choices=None, help=None, metavar=None)

In [35]:
args = parser.parse_args(["--dataset_path", f"{str(dataset_path)}", "--emission_matrix_path", f"{str(emission_matrix_path)}"])


In [36]:
args

Namespace(batch_size=1, cuda=False, dataset_path='\\nic\\sequences.npy', emission_matrix_path='\\nic\\emission_matrix.npy', hidden_dim=16, jit=False, learning_rate=0.05, nn_channels=2, nn_dim=48, num_steps=100, print_shapes=False, raftery_parameterization=False, seed=0, time_compilation=False, tmc=False, tmc_num_samples=10, truncate=None)

In [37]:
svi, model, guide, sequences, lengths, emission_matrix = main(args)

 Trace Shapes:              
  Param Sites:              
 Sample Sites:              
  probs_x dist         | 7 7
         value         | 7 7
    tones dist         |    
         value       7 |    
sequences dist         |    
         value     172 |    
    x_0_0 dist         |    
         value   7   1 |    
    y_0_0 dist   7   7 |    
         value         |    
    x_0_1 dist   7   1 |    
         value 7 1   1 |    
    y_0_1 dist 7 1   7 |    
         value         |    
    x_0_2 dist 7 1   1 |    
         value   7   1 |    
    y_0_2 dist   7   7 |    
         value         |    
    x_0_3 dist   7   1 |    
         value 7 1   1 |    
    y_0_3 dist 7 1   7 |    
         value         |    
    x_0_4 dist 7 1   1 |    
         value   7   1 |    
    y_0_4 dist   7   7 |    
         value         |    
    x_0_5 dist   7   1 |    
         value 7 1   1 |    
    y_0_5 dist 7 1   7 |    
         value         |    
    x_0_6 dist 7 1   1 |    
         value

	    0	5.906041628815407
	Training step: 1
	    1	5.866538540152616
	Training step: 2
	    2	5.8287614689316865
	Training step: 3
	    3	5.792788767260174
	Training step: 4
	    4	5.758686296329942
	Training step: 5
	    5	5.726500045421512
	Training step: 6
	    6	5.696240802143896
	Training step: 7
	    7	5.667865416061047
	Training step: 8
	    8	5.641312000363372
	Training step: 9
	    9	5.616493118640989
	Training step: 10
	   10	5.593320198946221
	Training step: 11
	   11	5.571693313953489
	Training step: 12
	   12	5.551518781795058
	Training step: 13
	   13	5.532711437136628
	Training step: 14
	   14	5.515202012172965
	Training step: 15
	   15	5.498919535792151
	Training step: 16
	   16	5.483801553415698
	Training step: 17
	   17	5.469798669149709
	Training step: 18
	   18	5.4568484284156975
	Training step: 19
	   19	5.444896893168605
	Training step: 20
	   20	5.433877634447675
	Training step: 21
	   21	5.423722519985465
	Training step: 22
	   22	5.414363417514535
	Training step

In [38]:
from pyro.infer import Predictive


num_samples = 1000
predictive = Predictive(model, guide=guide, num_samples=num_samples)


In [48]:
svi_samples = {k: v.detach().cpu().numpy()
           for k, v in predictive(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size, include_prior=False).items()
           if k != "obs"}

In [49]:
dir(svi)

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_categorical',
 '_idx_by_chain',
 '_reset',
 '_traces',
 'chain_ids',
 'evaluate_loss',
 'exec_traces',
 'guide',
 'information_criterion',
 'log_weights',
 'loss',
 'loss_and_grads',
 'marginal',
 'model',
 'num_chains',
 'num_samples',
 'num_steps',
 'optim',
 'run',
 'step']

In [41]:
svi_samples["probs_x"].shape

(1000, 1, 7, 7)

In [42]:
translation_matrix = np.mean(svi_samples["probs_x"], axis=0).reshape((7,7))

In [43]:
pyro.get_param_store()

In [44]:
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}

In [45]:
confusion_matrix_test_table = pd.DataFrame(
data=translation_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,0.522551,0.005990,0.291232,0.005623,0.006234,0.161763,0.006605
Oriented,0.004407,0.726990,0.015010,0.016219,0.003330,0.229130,0.004908
Precieved_Not_Oriented,0.293806,0.015000,0.313824,0.011298,0.004338,0.353251,0.008478
Precieved_Oriented,0.004289,0.180034,0.007618,0.036539,0.003174,0.763774,0.004582
Slanted,0.060348,0.004428,0.610174,0.015538,0.013559,0.278881,0.017069
Unidentified,0.079511,0.179855,0.225463,0.019644,0.003299,0.485070,0.007163
No_sample,0.226838,0.273374,0.148346,0.003117,0.002603,0.264377,0.081344


In [46]:
test_class_count = confusion_matrix_test_table.sum(axis=1)
test_class_count

Not_Oriented              0.999999
Oriented                  0.999994
Precieved_Not_Oriented    0.999996
Precieved_Oriented        1.000009
Slanted                   0.999997
Unidentified              1.000007
No_sample                 0.999999
dtype: float32

In [47]:
sequences[0,:]

tensor([2, 4, 1, 4, 4, 3, 3, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1],
       dtype=torch.int32)

In [50]:
translation_matrix = np.mean(svi_samples["probs_x"], axis=0).reshape((7,7))

In [51]:
confusion_matrix_test_table = pd.DataFrame(
data=translation_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,0.522551,0.005990,0.291232,0.005623,0.006234,0.161763,0.006605
Oriented,0.004407,0.726990,0.015010,0.016219,0.003330,0.229130,0.004908
Precieved_Not_Oriented,0.293806,0.015000,0.313824,0.011298,0.004338,0.353251,0.008478
Precieved_Oriented,0.004289,0.180034,0.007618,0.036539,0.003174,0.763774,0.004582
Slanted,0.060348,0.004428,0.610174,0.015538,0.013559,0.278881,0.017069
Unidentified,0.079511,0.179855,0.225463,0.019644,0.003299,0.485070,0.007163
No_sample,0.226838,0.273374,0.148346,0.003117,0.002603,0.264377,0.081344


In [ ]:
# Dataset loading
def load_sequences_np(dataset_path):
    sequences = {"train": {}}
    
#     sequence_list = [torch.Tensor(x) for x in np.load(str(dataset_path))]
    
    sequences_tensor = np.load(str(dataset_path))
    
    
    sequences["train"]["sequence_lengths"] = np.array([len(sequence) for sequence in sequences_tensor])
    sequences["train"]["sequences"] = sequences_tensor
    return sequences

In [ ]:
data = load_sequences_np(args.dataset_path)

In [ ]:
sequences = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [ ]:
sequences.reshape(-1)

In [ ]:
lengths

In [ ]:
emission_matrix_np = np.load(args.emission_matrix_path)


In [ ]:
from hmmlearn import hmm

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=10000, params="st", init_params="st")

In [ ]:
model.emissionprob_ = emission_matrix_np

In [ ]:
model.fit(sequences.reshape(-1, 1), lengths)

In [ ]:
model.emissionprob_

In [ ]:
model.transmat_

In [ ]:
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=50, params="st", init_params="st")
model.emissionprob_ = emission_matrix_np
model.fit(sequences.reshape(-1, 1), lengths)
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=500, params="st", init_params="st")
model.emissionprob_ = emission_matrix_np
model.fit(sequences.reshape(-1, 1), lengths)
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
transition_matrix = np.array([[9.17269248e-001, 6.53003806e-118, 8.23362715e-002,
        7.72427778e-085, 1.53698963e-014, 3.43175139e-011,
        3.94480110e-004],
       [1.49172147e-161, 8.73311971e-001, 1.82502958e-049,
        1.26687948e-001, 8.14069900e-008, 1.46260187e-014,
        1.51752796e-026],
       [1.41727846e-003, 1.18840441e-019, 9.49950247e-001,
        1.82914991e-003, 9.07768488e-003, 2.60561958e-002,
        1.16694438e-002],
       [6.51786889e-084, 1.39464295e-002, 1.01244429e-018,
        8.97187712e-001, 8.36640492e-002, 4.86750640e-005,
        5.15313390e-003],
       [6.15269241e-003, 2.86600087e-010, 3.19099140e-002,
        4.01633777e-002, 9.05916897e-001, 1.58571186e-002,
        1.00840900e-032],
       [3.72329494e-005, 5.49847908e-003, 2.38550441e-002,
        3.82597321e-003, 7.01993013e-002, 8.82466367e-001,
        1.41176022e-002],
       [3.63508204e-009, 4.16695519e-002, 1.42071041e-001,
        2.19540222e-032, 6.30280002e-029, 1.49592737e-001,
        6.66666667e-001]])
print(f"HMM transitions")
print(f"Oriented to slanted: {transition_matrix[1, 4] + transition_matrix[3,4]}")
print(f"slanted to oriented: {transition_matrix[4, 1] + transition_matrix[4,3]}")
print(f"unoriented to slanted: {transition_matrix[0, 4] + transition_matrix[2,4]}")
print(f"slanted to unoriented: {transition_matrix[4, 0] + transition_matrix[4,2]}")


In [ ]:
sequences[1, :]

In [ ]:
model.decode(sequences[1,:].reshape(-1,1))

In [ ]:
sequences[0, :]

In [ ]:
model.decode(sequences[0,:].reshape(-1,1))

In [ ]:
naive_transition_matrix = np.zeros((7,7))
for sequence in sequences:
    prev_state = 6
    for state in sequence:
#         print(state)
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state

In [ ]:

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
naive_transition_matrix = naive_transition_matrix / np.sum(naive_transition_matrix, axis=1).reshape((-1, 1))
print(naive_transition_matrix.sum(axis=1))

confusion_matrix_test_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table 

In [ ]:
transition_matrix = naive_transition_matrix
print(f"Naive CNN transitions")
print(f"Oriented to slanted: {transition_matrix[1, 4] + transition_matrix[3,4]}")
print(f"slanted to oriented: {transition_matrix[4, 1] + transition_matrix[4,3]}")
print(f"unoriented to slanted: {transition_matrix[0, 4] + transition_matrix[2,4]}")
print(f"slanted to unoriented: {transition_matrix[4, 0] + transition_matrix[4,2]}")


In [ ]:
X, Z = model.sample(100)

In [ ]:
X

In [ ]:
Z